<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/294_HITL_Routing_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Routing decision utilities for HITL Orchestrator

In [ ]:
"""Routing decision utilities for HITL Orchestrator"""

from typing import Dict, Any, Optional, List


def apply_routing_policy(
    risk_level: str,
    confidence_score: float,
    routing_policy: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Apply routing policy to determine routing decision for a task.

    Args:
        risk_level: Task risk level ("low", "medium", "high")
        confidence_score: Agent confidence score (0.0-1.0)
        routing_policy: Routing policy dictionary with rules

    Returns:
        Routing decision dictionary with:
        - routing_decision: "auto_approve" | "human_review" | "escalate"
        - assigned_human_role: Optional[str]
        - rule_applied: str (rule_id)
        - reasoning: str
    """
    rules = routing_policy.get("rules", [])

    # Sort rules by priority (lower number = higher priority)
    sorted_rules = sorted(rules, key=lambda r: r.get("priority", 999))

    # Find first matching rule
    for rule in sorted_rules:
        conditions = rule.get("conditions", {})

        # Check risk level condition
        if "risk_level" in conditions:
            if risk_level != conditions["risk_level"]:
                continue

        # Check confidence condition
        if "min_confidence" in conditions:
            min_confidence = conditions["min_confidence"]
            if confidence_score < min_confidence:
                continue

        # Rule matches - apply it
        action = rule.get("action")
        assigned_role = rule.get("assigned_human_role")
        rule_id = rule.get("rule_id", "unknown")

        # Build reasoning
        reasoning_parts = []
        if "risk_level" in conditions:
            reasoning_parts.append(f"Risk level: {risk_level}")
        if "min_confidence" in conditions:
            reasoning_parts.append(f"Confidence: {confidence_score:.2f} >= {conditions['min_confidence']}")
        reasoning = ", ".join(reasoning_parts) if reasoning_parts else "Default rule"

        return {
            "routing_decision": action,
            "assigned_human_role": assigned_role,
            "rule_applied": rule_id,
            "reasoning": reasoning
        }

    # No rule matched - default to human review
    return {
        "routing_decision": "human_review",
        "assigned_human_role": "domain_reviewer",
        "rule_applied": "default",
        "reasoning": "No matching rule found, defaulting to human review"
    }


def make_routing_decision(
    task_id: str,
    risk_level: str,
    confidence_score: float,
    routing_policy: Dict[str, Any]
) -> Dict[str, Any]:
    """
    Make routing decision for a single task.

    Args:
        task_id: Task identifier
        risk_level: Task risk level
        confidence_score: Agent confidence score
        routing_policy: Routing policy

    Returns:
        Complete routing decision dictionary
    """
    decision = apply_routing_policy(risk_level, confidence_score, routing_policy)

    return {
        "task_id": task_id,
        "risk_level": risk_level,
        "confidence_score": confidence_score,
        **decision
    }

